In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 30)

%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.


In [15]:
df = pd.read_csv("../input/creditcard.csv")
df = df.drop(columns=["Time"])
df


In [16]:
columns = df.columns
print(columns)
print(df.shape)

In [17]:
class_counts = pd.value_counts(df['Class'], sort=True)
class_counts = pd.DataFrame(class_counts).reset_index()
print(class_counts)
f, ax = plt.subplots(figsize=(10, 8))
sns.barplot(x='index', y='Class', data=class_counts, ax=ax)


Apparently,  it is an extremely unbalanced data. if we simply apply our classify model to this data without any other data processing,  it will be very hard for us to get an ideal result. There ars serval solutions for this kind of  unbalanced data:
1. UNDER-sampling,
2. OVER-sampling
3. Synthetic Minority Over-Sampling Technique

In this notebook, I would like to use UNDER-sampling as it is the simplest. Later, I will try to use Synthetic Minority Over-Sampling Technique and compare it with UNDER-sampling



In [18]:
corr = abs(df.corr())
corr = corr.nlargest(n=10, columns='Class')
columns = list(corr.index)
print(columns)
corr = corr[corr.index]
f, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(corr, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=corr.columns, xticklabels=corr.index, ax=ax)


1. I calculate the correlation bewteen each feature  and select ten features depends on the coefficient to feature 'Class'. It can be foubd that other feathures only has correlation with feature 'Class'  and has nothing to do with each others. As this dataset has been already handled by PCA, the features displayed here is dependent. So, there is no need to do more feature selection in this case.
2. Next, I will plot a boxplot to display the difference of each feature according the their class.

In [19]:
target = 'Class'
variables = [item for item in columns if item != 'Class']
variables = np.reshape(variables, [3, 3])
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(18, 18))
for i in range(3):
    for j in range(3):
        sns.boxplot(x=target, y=variables[i, j], data=df, ax=axes[i, j])


In [78]:
from scipy.stats import norm
from scipy import stats
sns.distplot(df['V17'], fit=norm);
fig = plt.figure()
res = stats.probplot(df['V17'], plot=plt)

From above I konw that for different classes, the features clearly has a different distributions. Commonly, we prefer to consider the variables in each column as a normal distributions, which is the base hypothesis of most mathine learning models. Apparently, V17 does not obey normal distribution, it has a clearly long tail.


In [49]:
from collections import Counter
features = df.columns
features = list(features)
features.remove("Class")
features
X, y = df[features], df['Class']
Counter(y)

In [50]:
from imblearn.over_sampling import SMOTE
X_resampled_smote, y_resampled_smote = SMOTE().fit_sample(X, y)
Counter(y_resampled_smote)

Using smote method, I create many samples belongs to Class 1, they are similar to the origin samples. Now the dataset has been balanced. Next, I will apply classfication algorithm to the dataset.

In [80]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_resampled_smote, y_resampled_smote,test_size=0.3, random_state=0)
Counter(y_test)

In [82]:
from sklearn.linear_model import LogisticRegression
c_param_range = [0.01,0.1,1,10,100]
for c in c_param_range:
    logistic = LogisticRegression(C=c)   
    logistic.fit(X_train, y_train)
    score = logistic.score(X_test, y_test)
    print('score at c=%s is %s' %(c, score, ))

Next,  I will caculate the confusion_matrix of our prediction, I chose c equal to 10 for example.

In [83]:
from sklearn.metrics import confusion_matrix
logistic = LogisticRegression(C=10)   
logistic.fit(X_train, y_train)
predict_test = logistic.predict(X_test)
c_matrix = confusion_matrix(y_test, predict_test)
c_matrix